In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import GPy
import itertools
%matplotlib inline
import math

In [3]:
data = pd.read_csv("../rawdata/raw_data_basis_2.csv")
df = data.loc[750:4250]
df.tail()

,Index,date,EEM_LAST,EEM_PREM,EEM_TR,SPY_LAST,SPY_PREM,SPY_TR,GOLD,VIX,...,lag_1_signal_5YSWAP_nDayRet_5_1_ret,lag_1_signal_HYG_TR_nDayRet_5_1_ret,lag_1_signal_OIL_nDayRet_5_1_ret,lag_1_signal_EEM_TR_nDayRet_20_1_ret,lag_1_signal_SPY_TR_nDayRet_20_1_ret,lag_1_signal_GOLD_nDayRet_20_1_ret,lag_1_signal_VIX_nDayRet_20_1_ret,lag_1_signal_5YSWAP_nDayRet_20_1_ret,lag_1_signal_HYG_TR_nDayRet_20_1_ret,lag_1_signal_OIL_nDayRet_20_1_ret
3617,3618,8/23/2017,44.53,0.678,56.5600,244.56,0.003,339.2590,107.7732,12.25,...,-0.027548,0.001139,0.001893,0.015349,-0.008003,0.015901,0.203606,-0.059367,-0.006250,-0.005220
3618,3619,8/24/2017,44.67,0.359,56.7378,243.99,-0.025,338.4683,108.3788,12.23,...,-0.029600,0.001138,0.034844,0.011816,-0.011599,-0.005357,0.276042,-0.052687,-0.007590,-0.006974
3619,3620,8/25/2017,44.93,0.672,57.0681,244.56,0.031,339.2590,107.9937,11.28,...,-0.000055,0.008243,0.007220,0.020561,-0.012985,0.024976,0.209693,-0.051559,-0.005340,-0.032830
3620,3621,8/28/2017,44.77,0.336,56.8648,244.57,-0.017,339.2729,112.2173,11.32,...,-0.012122,0.008574,-0.013193,0.026972,-0.009518,0.003309,0.096210,-0.049740,-0.003644,-0.037015
3621,3622,8/29/2017,44.69,0.517,56.7632,244.85,0.000,339.6613,112.4835,11.70,...,-0.014887,0.007652,-0.016888,0.022145,-0.008915,0.045447,0.103314,-0.063427,-0.003531,-0.071756


In [4]:
### Given features and targets make an augmented structure

#make a flat augmented structure, this time
max_delta = 4

#adapt the functional augmented to just do augmented, ie set year =1
years = 1
days = df.shape[0]
n=list(range(0,years))
ob=list(range(0,days))
c = list(itertools.product(n, ob))
tar=list(range(0,days))
combo = list(itertools.product(c,tar))
flat_combo = [[elem[0][0],elem[0][1],elem[1]] for elem in combo]

aug_raw_data = pd.DataFrame(flat_combo, columns=['Year', 'Ob_day', 'Tar_day',])
aug_raw_data['Delta']= aug_raw_data['Tar_day'] - aug_raw_data['Ob_day']

#do the features - messy way but have adapted from func-aug
#now put in the features
ob_pd = data #.shift(1)
features = ob_pd[['signal_SPY_TR_nDayRet_20_1_ret','VIX','signal_VIX_nDayRet_20_1_ret']][750:4251]
features = features.values
#features[0]=0
oprices = np.tile(features[:,0].reshape(years,days),days).reshape((-1,))
aug_raw_data['SPY_sig1'] = oprices.reshape(days,days).T.flatten()

ovols = np.tile(features[:,1].reshape(years,days),days).reshape((-1,))
osig2 = np.tile(features[:,2].reshape(years,days),days).reshape((-1,))
aug_raw_data['SPY_sig1'] = oprices.reshape(days,days).T.flatten()
aug_raw_data['VIX'] = ovols.reshape(days,days).T.flatten()
aug_raw_data['SPY_sig2'] = osig2.reshape(days,days).T.reshape((-1,))

prices = data.loc[750:4250,"EEM_PREM"]

#do the targets
aug_raw_data['TPrices'] = np.tile(prices.reshape(years,days),days).reshape((-1,))

aug_raw_data = aug_raw_data[(aug_raw_data["Delta"] >= 0) & (aug_raw_data["Delta"] <= max_delta)]
aug_raw_data = aug_raw_data.reset_index(drop=True)
aug_raw_data.drop('Year', axis=1, inplace=True)
aug_raw_data.drop('Tar_day', axis=1, inplace=True)

 /usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:37: FutureWarning:reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [5]:
#split the data between training and test
def make_data(pred_day,days_back):
    train_data = aug_raw_data[(aug_raw_data['Ob_day']<pred_day) & ((aug_raw_data['Ob_day']+aug_raw_data['Delta'])<pred_day) ]

    #test data some delta zeros
    delta_zero = aug_raw_data[(aug_raw_data['Delta']==0) & (aug_raw_data['Ob_day']<pred_day) & 
                             (aug_raw_data['Ob_day']>=(pred_day-days_back))]
    test_data = pd.concat([delta_zero,aug_raw_data[(aug_raw_data['Ob_day']==pred_day)]])

    #train data no longer doing sampling
    lookback = 500
    samp_aug_train = train_data[-lookback:]

    #so now have training data in augmented form samp_aug_train and test data - test_data
    #optimisation
    feat_lst = ['Ob_day','Delta','VIX'] #'SPY_sig1','SPY_sig2','VIX']

    X = (samp_aug_train[feat_lst].as_matrix())
    y = ((samp_aug_train[['TPrices']]).as_matrix())

    Xpred = (test_data[feat_lst].as_matrix())
    ytest = ((test_data[['TPrices']]).as_matrix())
    return X,y,Xpred,ytest


In [6]:
def do_opt(restarts,X,y,k):
    mod = GPy.models.GPRegression(X, y, k)
    mod.constrain_positive('')
    mod.optimize(messages=True)
    mod.optimize_restarts(num_restarts=restarts)  
    return mod
    

In [31]:
Xpred

array([[  2.86800000e+03,   0.00000000e+00,   1.22300000e+01],
       [  2.86800000e+03,   1.00000000e+00,   1.22300000e+01],
       [  2.86800000e+03,   2.00000000e+00,   1.22300000e+01],
       [  2.86800000e+03,   3.00000000e+00,   1.22300000e+01]])

In [32]:
feat_lst = ['Ob_day','Delta','VIX'] #'SPY_sig1','SPY_sig2','VIX']
k_input_d = len(feat_lst)
k = GPy.kern.OU(input_dim=k_input_d,ARD=True)# + GPy.kern.White(input_dim=k_input_d)

In [33]:
#pred_day =500
days_back =0 #this is for test data
first= 500
last = 3495
pred_mean = pd.DataFrame(columns=['Ind','D0','D1','D2','D3','D4'])
pred_var = pd.DataFrame(columns=['Ind','D0','D1','D2','D3','D4'])
pred_market = pd.DataFrame(columns=['Ind','Price'])

for pred_day in range(first,last):
    ind = pred_day-500
    X,y,Xpred,ytest = make_data(pred_day, days_back)

    #if we are optimising
    opt = not(pred_day%10)
    if opt:
        restarts =3
        mod = do_opt(restarts,X,y,k)  
        ypred, var = mod.predict(Xpred)
        gaussian_noise_var = mod.Gaussian_noise.variance

    #if we are not optimising
    else:
        X,y,Xpred,ytest = make_data(pred_day, days_back)

        mod = GPy.models.GPRegression(X, y, k)
        mod.Gaussian_noise.variance=gaussian_noise_var
        ypred, var = mod.predict(Xpred)

    #populate preds
    pred_mean.loc[ind,'Ind'] = pred_day+751
    pred_mean.loc[ind,1:]= ypred.flatten()

    pred_var.loc[ind,'Ind'] = pred_day+751
    pred_var.loc[ind,1:]= var.flatten()
    
    pred_market.loc[ind,'Ind'] = pred_day+751
    pred_market.loc[ind,'Price']= ytest.flatten()[0]


Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s08  0003   7.870355e+02   6.100859e+01 
    00s17  0006   7.839342e+02   5.323489e+01 
    01s37  0049   7.767373e+02   1.748633e-04 
Runtime:     01s37
Optimization status: Converged

Optimization restart 1/3, f = 776.7372555802565


 /usr/local/lib/python3.4/dist-packages/paramz/transformations.py:109: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/3, f = 776.7372391316283
Optimization restart 3/3, f = 776.7372356579189
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s05  0002   7.883961e+02   7.228245e+03 
    00s17  0006   7.595423e+02   3.173996e-01 
    00s45  0016   7.594946e+02   8.616589e-11 
Runtime:     00s45
Optimization status: Converged

Optimization restart 1/3, f = 759.4945854830955
Optimization restart 2/3, f = 759.4945890824918
Optimization restart 3/3, f = 759.4945866824794
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s08  0003   7.087355e+02   8.602502e+01 
    00s17  0006   7.081402e+02   5.097968e-01 
    00s45  0016   7.081247e+02   2.131212e-07 
Runtime:     00s45
Optimization status: Converged

Optimization restart 1/3, f = 708.1246867536397
Optimization restart 2/3, f = 708.1246866521461
Optimization restart 3/3, f = 708.1246861383585
Running L-BFGS-B (Scipy implementation) Code:
  

Optimization restart 3/3, f = 1175.5591049897405
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s11  0004   1.220016e+03   1.953321e+03 
    00s51  0018   1.181337e+03   1.740290e-08 
Runtime:     00s51
Optimization status: Converged

Optimization restart 1/3, f = 1181.3367583479967
Optimization restart 2/3, f = 1181.336864930829
Optimization restart 3/3, f = 1181.3367688829155
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s05  0002   1.353229e+03   2.839696e+04 
    00s50  0018   1.174669e+03   1.853020e-07 
Runtime:     00s50
Optimization status: Converged

Optimization restart 1/3, f = 1174.6692030919999
Optimization restart 2/3, f = 1174.6692084627691
Optimization restart 3/3, f = 1174.6692044397985
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s14  0005   1.170180e+03   3.000176e+02 
    00s51  0018   1.159632e+03   1.833591e-07

 /usr/local/lib/python3.4/dist-packages/GPy/kern/src/stationary.py:158: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.4/dist-packages/GPy/kern/src/stationary.py:129: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.4/dist-packages/GPy/kern/src/stationary.py:130: RuntimeWarning:invalid value encountered in add


...

Optimization restart 2/3, f = 382.1556492088
Optimization restart 3/3, f = 431.2702167282209
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s17  0006   4.110465e+02   1.614488e+01 
    01s72  0061   4.066188e+02   8.865708e-09 
Runtime:     01s72
Optimization status: Converged

Optimization restart 1/3, f = 406.6187623810331
Optimization restart 2/3, f = 406.61880449068747
Optimization restart 3/3, f = 406.61876390157397
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s08  0003   1.420370e+03   1.017070e+06 
    00s17  0006   4.190286e+02   1.475883e+01 
    00s72  0026   4.186010e+02   6.387729e-08 
Runtime:     00s72
Optimization status: Converged

Optimization restart 1/3, f = 418.6009662803394
Optimization restart 2/3, f = 418.6009711354392
Optimization restart 3/3, f = 418.60096765700735
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
   

Optimization restart 2/3, f = 499.02008458840146
Optimization restart 3/3, f = 499.0200828159108
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s00  0000   6.149792e+02           nan 
    00s17  0006   5.315733e+02   3.606215e+00 
    00s51  0018   5.315143e+02   6.859693e-07 
Runtime:     00s51
Optimization status: Converged

Optimization restart 1/3, f = 531.5142663597131
Optimization restart 2/3, f = 531.5138851507679
Optimization restart 3/3, f = 531.5138850126641
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s08  0003   5.730691e+02   4.794279e+03 
    00s17  0006   5.550109e+02   5.338288e+00 
    00s48  0017   5.548826e+02   6.060706e-07 
Runtime:     00s48
Optimization status: Converged

Optimization restart 1/3, f = 554.8825526596696
Optimization restart 2/3, f = 554.8825368668299
Optimization restart 3/3, f = 554.8825362274583
Running L-BFGS-B (Scipy implementation) Code:
  

    00s14  0005   4.635794e+02   6.348518e+01 
    00s17  0006   4.633406e+02   1.611341e+00 
    01s87  0067   4.632350e+02   3.913474e-08 
Runtime:     01s87
Optimization status: Converged

Optimization restart 1/3, f = 463.23503799041276
Optimization restart 2/3, f = 463.23503799050707
Optimization restart 3/3, f = 463.235040623654
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s17  0006   4.429477e+02   2.303088e+00 
    01s04  0037   4.428397e+02   3.201629e-06 
Runtime:     01s04
Optimization status: Converged

Optimization restart 1/3, f = 442.83966573489306
Optimization restart 2/3, f = 442.8396657315045
Optimization restart 3/3, f = 442.8396657562685
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s15  0004   4.245279e+02   1.071483e+03 
    00s18  0005   4.217825e+02   5.086511e+02 
    01s35  0047   4.199635e+02   3.023355e-10 
Runtime:     01s35
Optimization status: Converge

Optimization restart 2/3, f = 308.2060093267856
Optimization restart 3/3, f = 308.2060271059482
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s08  0003   1.431993e+03   1.425816e+06 
    00s17  0006   2.755666e+02   5.466218e+01 
    00s64  0023   2.753902e+02   3.275819e-07 
Runtime:     00s64
Optimization status: Converged

Optimization restart 1/3, f = 275.3902141610875
Optimization restart 2/3, f = 275.3902257040192
Optimization restart 3/3, f = 275.3902134508234
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s17  0006   2.782902e+02   5.582523e+01 
    01s02  0037   2.778750e+02   6.388981e-06 
Runtime:     01s02
Optimization status: Converged

Optimization restart 1/3, f = 277.8749618419031
Optimization restart 2/3, f = 277.8749741193714
Optimization restart 3/3, f = 277.8749591923247
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
   

ValueError: could not broadcast input array from shape (4) into shape (5)

In [34]:
pred_mean[:-1].tail()

,Ind,D0,D1,D2,D3,D4
2363,3614,0.204472,0.2102,0.210342,0.206607,0.200233
2364,3615,-0.153976,-0.156709,-0.155344,-0.152122,-0.147947
2365,3616,-0.0333015,-0.0336416,-0.0339183,-0.0338941,-0.0336963
2366,3617,0.0486441,0.0502097,0.0500835,0.0489393,0.0471433
2367,3618,0.183236,0.187606,0.187963,0.185535,0.181144


In [35]:
pred_var.tail()

,Ind,D0,D1,D2,D3,D4
2363,3614,0.142965,0.142872,0.143394,0.144329,0.145539
2364,3615,0.142965,0.142872,0.143394,0.144329,0.145539
2365,3616,0.142965,0.142872,0.143394,0.144329,0.145539
2366,3617,0.142965,0.142872,0.143394,0.144329,0.145539
2367,3618,0.142965,0.142872,0.143394,0.144329,0.145539


In [38]:
pred_market.head()

,Ind,Price
0,1251,1.744
1,1252,0.311
2,1253,0.888
3,1254,0.26
4,1255,0.239


In [37]:
pred_mean[:-1].to_csv('../results/jpred_eem_mean.csv',index=False,header=False)
pred_var.to_csv('../results/jpred_eem_var.csv',index=False,header=False)
pred_market.to_csv('../results/jpred_eem_market.csv',index=False,header=False)

In [23]:
data1 = pd.read_csv("../rawdata/raw_data.csv")

In [25]:
data1.head()

,Index,date,IWM,SPY,XLE,XLF,XLI,XLK,XLP,XLU,...,XLP_LOW,XLU_LOW,XLV_LOW,XLY_LOW,GOLD_LOW,IWM_VOLUME,IWM_LOW,IWM_HIGH,IWM_LAST,IWM_OPEN
0,1,5/26/2000,45.7188,138.3483,30.9978,19.7635,28.8411,47.1875,24.3913,28.2699,...,24.0625,27.7188,27.9688,26.2813,56.58,76800,45.2813,45.7188,45.7188,45.5313
1,2,5/30/2000,47.4063,142.8596,31.4368,20.1710,29.3578,50.2500,24.2502,28.4745,...,23.8438,27.9375,28.2813,26.1406,57.14,57600,46.3750,47.4063,47.4063,46.3750
2,3,5/31/2000,47.5781,143.1729,32.3776,20.4129,29.5614,49.2813,24.0150,28.6319,...,23.9375,28.1250,29.1250,26.5000,56.25,36000,47.5625,48.1875,47.5781,47.5625
3,4,6/1/2000,48.6563,145.6792,32.0797,21.0878,29.7962,51.5000,24.0777,28.8365,...,23.4063,28.2656,29.2500,27.0000,56.28,7000,48.5547,48.6563,48.6563,48.5547
4,5,6/2/2000,51.1875,148.2169,30.9821,21.9537,30.0154,53.8438,23.2469,28.7421,...,23.1250,28.3906,29.8125,27.7500,57.99,29400,50.8594,51.1875,51.1875,50.8594


In [28]:
df=pd.DataFrame(index=data1.index)